This file takes in a user-defined class with a certain name. Instantiating the network is simply for testing purposes

In [1]:
import importlib.util
import os
import torch
from torch import nn
from torch.utils.data import DataLoader

In [21]:
class NetworkAnalyzer:

    def __init__(self, model_architecture: nn.Module, amount_to_produce: int, success_criteria: callable = None, max_attempts: float = None):
        """
        Initializes the NetworkAnalyzer class with the required model architecture and configuration.

        Args:
            model_architecture (nn.Module): A PyTorch neural network model class specifying the architecture.
            amount_to_produce (int): Number of good neural networks to generate based on success criteria.
            success_criteria (callable, optional): A user-defined callable or lambda function specifying the 
                                                   acceptable loss threshold for a successful model. Defaults to 1.0.
            max_attempts (float, optional): Maximum number of attempts to produce a successful network, defaults
                                            to 130% of `amount_to_produce` if not specified.
        """
        self.model_architecture = model_architecture
        self.amount_to_produce = amount_to_produce
        self.success_criteria = success_criteria if success_criteria is not None else self.default_success_criteria()
        self.max_attempts = max_attempts if max_attempts is not None else self.default_max_attempts()

    def default_success_criteria(self):
        """
        Provides a default success criteria if none is specified by the user.
        The default is a loss value of 1.0.

        Returns:
            float: The default success criteria (loss value).
        """
        return 1.0

    def default_max_attempts(self):
        """
        Sets the maximum number of attempts to produce successful networks to 130% of the amount requested,
        if not specified by the user.

        Returns:
            int: The calculated maximum number of attempts.
        """
        return int(self.amount_to_produce * 1.3)

    def show_loss(self, epoch: int, loss_value: float):
        """
        Prints the loss value for a specific epoch during training.

        Args:
            epoch (int): The current epoch number during training.
            loss_value (float): The loss value for the current epoch.
        """
        print(f"Epoch [{epoch+1}], Loss: {loss_value:.4f}")

    def train_network(self, model: nn.Module, train_loader: DataLoader, num_epochs: int, optimizer, loss_fn: torch.nn.Module):
        """
        Trains a single network model over a specified number of epochs.

        Args:
            model (nn.Module): The PyTorch neural network model to train.
            train_loader (DataLoader): The DataLoader for the training dataset.
            num_epochs (int): Number of epochs to train the model.
            optimizer (Optimizer): The optimizer used to update model weights.
            loss_fn (torch.nn.Module): The loss function used to compute the training loss.
        """
        for epoch in range(num_epochs):
            print(f"Epoch {epoch+1}/{num_epochs}")
            for batch in train_loader:
                input, target = batch
                optimizer.zero_grad()           
                output = model(input)           
                loss = loss_fn(output, target)  
                loss.backward()                 
                optimizer.step()                
            self.show_loss(epoch, loss.item())  # Print loss per epoch
            model.final_loss = loss.item()      # Save the final loss in the model

    def generate_networks(self, train_loader: DataLoader, test_loader: DataLoader, num_epochs: int, loss_fn: torch.nn.Module):
        """
        Generates multiple neural networks and trains them until a successful number of networks is produced,
        based on the success criteria and the maximum number of allowed attempts.

        Args:
            train_loader (DataLoader): The DataLoader for the training dataset.
            test_loader (DataLoader): The DataLoader for the testing dataset (unused here but included for completeness).
            num_epochs (int): The number of epochs for which each network should be trained.
            loss_fn (torch.nn.Module): The loss function to use during training.
        """
        self.models = []  # List to store successfully trained models

        # Ensure directories for saving networks exist
        working_dir = "Working Networks"
        broken_dir = "Broken Networks"

        if not os.path.exists(working_dir):
            os.makedirs(working_dir)  # Create directory if it doesn't exist
        if not os.path.exists(broken_dir):
            os.makedirs(broken_dir)


        attempt = 0  # Counter for total attempts
        success_count = 0  # Counter for successful networks

        while success_count < self.amount_to_produce:
            print(f"Training Network {attempt+1}")

            # Stop if maximum attempts are reached
            if attempt >= self.max_attempts:
                print("Error: Maximum number of attempts reached without meeting success criteria.")
                break

            # Instantiate a new network
            network_to_be = self.model_architecture()
            optimizer = torch.optim.SGD(network_to_be.parameters(), lr=0.001, momentum=0.9)  # SGD optimizer
            
            # Train the network
            self.train_network(network_to_be, train_loader, num_epochs, optimizer, loss_fn)

            # Check if the network meets the success criteria
            if network_to_be.final_loss <= self.success_criteria:
                torch.save(network_to_be.state_dict(), os.path.join(working_dir, f'network_{attempt+1}.pt'))
                self.models.append(network_to_be)  # Add successful model to the list
                success_count += 1
            else:
                torch.save(network_to_be.state_dict(), os.path.join(broken_dir, f'network_{attempt+1}.pt'))

            attempt += 1  # Increment attempt counter

        if success_count == self.amount_to_produce:
            print(f"Successfully trained {success_count} networks.")
        else:
            print(f"{len(self.models)} successful networks created out of {self.amount_to_produce}.")

### Everyrthing past here is testing. Will be deleted before pushing on to main branch

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [19]:
batch_size = 30

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [6]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

import torch.optim as optim

criterion = nn.CrossEntropyLoss()



In [23]:
analyzer = NetworkAnalyzer(Net,2, max_attempts=3)
analyzer.generate_networks(trainloader, testloader, 5, criterion)


Training Network 1
Epoch 1/5
Epoch [1], Loss: 1.8357
Epoch 2/5
Epoch [2], Loss: 1.5371
Epoch 3/5
Epoch [3], Loss: 1.4437
Epoch 4/5
Epoch [4], Loss: 1.4822
Epoch 5/5
Epoch [5], Loss: 1.3012
Training Network 2
Epoch 1/5
Epoch [1], Loss: 2.0784
Epoch 2/5
Epoch [2], Loss: 1.6727
Epoch 3/5
Epoch [3], Loss: 1.8037
Epoch 4/5
Epoch [4], Loss: 1.4079
Epoch 5/5
Epoch [5], Loss: 1.2976
Training Network 3
Epoch 1/5
Epoch [1], Loss: 2.0482
Epoch 2/5
Epoch [2], Loss: 1.4924
Epoch 3/5
Epoch [3], Loss: 1.3063
Epoch 4/5
Epoch [4], Loss: 1.3851
Epoch 5/5
Epoch [5], Loss: 0.9773
Training Network 4
Error: Maximum number of attempts reached without meeting success criteria.
1 successful networks created out of 2.
